In [24]:
import findspark

import pandas
import plotly as p
import plotly.plotly as py
from plotly.graph_objs import *
import plotly.figure_factory as ff
import plotly.plotly as py
import plotly.graph_objs as go
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

findspark.init("C:\spark-2.2.1-bin-hadoop2.7")
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.types import *
sparkSession = SparkSession.builder.appName(
    "option-pricer-write-to-hadoop").getOrCreate()

In [29]:


p.tools.set_credentials_file(username='lalor188', api_key='MQxJd3Rs9X5aKOZiDEgt')

resultsHiveDF = sparkSession.read.format('json').load('E:\ProjectDB')

resultsHiveDF.createOrReplaceTempView("optionData")

tickers = sparkSession.sql("SELECT Ticker FROM optionData").collect()


expDate = sparkSession.sql("SELECT ExpirationDate FROM optionData WHERE Ticker Like 'MMM'").collect()

riskFreeRates = sparkSession.sql("SELECT riskFreeRates FROM optionData WHERE Ticker Like 'MMM'").collect()

strikePrices = sparkSession.sql("SELECT Prices['StrikePrice'] FROM optionData WHERE Ticker Like 'ZTS'").collect()

numOfDays = sparkSession.sql("SELECT NumberOfDays FROM optionData WHERE Ticker Like 'ZTS'").collect()



In [114]:
# Cell to declare all funcions

# Function to get prices based on a specific ticker symbol
def getOptionPricesByTicker(symbol):
    return sparkSession.sql("SELECT Prices FROM optionData WHERE Ticker Like '" + symbol + "'").collect()

# Function to turn int to array of ints for number of days before expDate
# eg. number of says is 5 returns [1,2,3,4,5]
def getNumberDaysArray(x):
    numDaysArr = []
    for i in range(1, x[0]['NumberOfDays'] +1):
        numDaysArr.append(i)
    return numDaysArr

def getPricesByStrike(optionPrices, strike):
    allPrices = []
    for i in optionPrices[0][0]:
        if i != None and i['StrikePrice'] == strike:
            allPrices.append(i[0])
    return allPrices

def getPricesByRiskFreeRate(optionPrices, RiskFreeRate):
    allPrices = []
    for i in optionPrices[0][0]:
        if i != None and i['RiskFreeRate'] == RiskFreeRate:
            allPrices.append(i[0])
    return allPrices


def getPricesByStrikeAndRate(optionPrices, strike, RiskFreeRate):
    allPrices = []
    for i in optionPrices[0][0]:
        if i != None and i['StrikePrice'] == strike and i['RiskFreeRate'] == RiskFreeRate:
            allPrices.append(i[0])
    return allPrices

def getCalls(optionPrices):
    callPrices = []
    for i in optionPrices[0][0]:
        if i['Call'] != None:
            callPrices.append(i['Call'])
    return callPrices

def getPuts(optionPrices):
    putPrices = []
    for i in optionPrices[0][0]:
        if i['Put'] != None:
            putPrices.append(i['Put'])
    return putPrices

def priceToArray(opPrices):
    prices = {}
    count = 1
    for i in opPrices:
        priceArr = []
        for x in i:
            if x != None:
                priceArr.append(x)
        prices['opt'+str(count)] = (priceArr.copy())
        count = count + 1
    return prices

In [109]:
prices = getOptionPricesByTicker('ZTS')

callPrices = []
putPrices = []

callPricesStrike = getCallPricesByStrike(prices, 85.0)

print(callPricesStrike)

callPricesRate = getCallPricesByRiskFreeRate(prices, 1.67)

callPricesSR = getCallPricesByStrikeAndRate(prices, 67.5, 1.67)

callPrices = getCalls(prices)

putPrices = getPuts(prices)
        
numberOfDays = getNumberDaysArray(numOfDays)
    

[Row(2018-03-11=0.0, 2018-03-12=0.0, 2018-03-13=0.0, 2018-03-14=0.0, 2018-03-15=0.0, 2018-03-16=0.0, 2018-03-17=None, 2018-03-18=None, 2018-03-19=None, 2018-03-20=None, 2018-03-21=None, 2018-03-22=None, 2018-03-23=None, 2018-03-24=None, 2018-03-25=None, 2018-03-26=None, 2018-03-27=None, 2018-03-28=None, 2018-03-29=None, 2018-03-30=None, 2018-03-31=None, 2018-04-01=None, 2018-04-02=None, 2018-04-03=None, 2018-04-04=None, 2018-04-05=None, 2018-04-06=None, 2018-04-07=None, 2018-04-08=None, 2018-04-09=None, 2018-04-10=None, 2018-04-11=None, 2018-04-12=None, 2018-04-13=None, 2018-04-14=None, 2018-04-15=None, 2018-04-16=None, 2018-04-17=None, 2018-04-18=None, 2018-04-19=None, 2018-04-20=None, 2018-04-21=None, 2018-04-22=None, 2018-04-23=None, 2018-04-24=None, 2018-04-25=None, 2018-04-26=None, 2018-04-27=None, 2018-04-28=None, 2018-04-29=None, 2018-04-30=None, 2018-05-01=None, 2018-05-02=None, 2018-05-03=None, 2018-05-04=None, 2018-05-05=None, 2018-05-06=None, 2018-05-07=None, 2018-05-08=None

In [112]:
data = []

opPriceDic = priceToArray(putPrices)

for key, values in opPriceDic.items():
    trace = go.Scatter(
        x = numberOfDays,
        y = values,
        name = 'option',
        line = dict(
            width = 4)
    )
    data.append(trace)
# Edit the layout
layout = dict(title = 'Average High and Low Temperatures in New York',
              xaxis = dict(title = 'Month'),
              yaxis = dict(title = 'Temperature (degrees F)'),
              )

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='styled-line')


In [87]:
import plotly.plotly as py
from plotly.graph_objs import *

import numpy as np

s = np.linspace(0, 2 * np.pi, 240)
t = np.linspace(0, np.pi, 240)
tGrid, sGrid = np.meshgrid(s, t)

r = 2 + np.sin(7 * sGrid + 5 * tGrid)  # r = 2 + sin(7s+5t)
x = r * np.cos(sGrid) * np.sin(tGrid)  # x = r*cos(s)*sin(t)
y = r * np.sin(sGrid) * np.sin(tGrid)  # y = r*sin(s)*sin(t)
z = r * np.cos(tGrid)                  # z = r*cos(t)

surface = Surface(x=x, y=y, z=z)
data = Data([surface])

layout = Layout(
    title='Parametric Plot',
    scene=Scene(
        xaxis=XAxis(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)'
        ),
        yaxis=YAxis(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)'
        ),
        zaxis=ZAxis(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)'
        )
    )
)

fig = Figure(data=data, layout=layout)
py.iplot(fig, filename='parametric_plot')